# Lab 5
## Data Structures & Algorithms
### Thursday, 7 March 2024

## Today

* [Brief introduction to databases](#databases)
* [Example: a blog page without a database](#blog)
* [Why Use a Database?](#why)
* [Databases in Flask](#databases_flask)
* [Example continued: a blog page (now with database)](#blog_database)
* [Exercises](#exercises)

## What are Databases? <a class="anchor" id="databases"></a>

Databases are organised collections of data, generally stored and accessed electronically from a computer system. Users interact with databases through database management systems. Databases are essential components of most software applications, providing a structured way to store, retrieve, and manage data efficiently. 

Databases can roughly be categorised into **two types**: 

1. **relational databases**: data in tables which have rows and columns, normally written and queried by using SQL
2. **NoSQL databases**: non-relational, use different query language, based on different data models

We will focus on relational databases!

## Example: a blog post without a database <a class="anchor" id="blog"></a>

Let's extend our existing Flask application, to include a blog page. Since our home page didn't include anything yet, we will turn this into the blog page. Initially, we'll hardcode the blog posts directly into our Flask app, and then we'll improve it by storing blog posts in a database. 

Our code will be following *roughly* [part 4](https://www.youtube.com/watch?v=cYWiDiIUxQc&list=PL-osiE80TeTs4UjLw5MM6OjgkjFeUxCYH&index=4) of the Flask tutorial (to which I linked a couple of weeks ago).

#### Step 1: Hardcoding Blog Posts

We'll start by hardcoding blog posts directly into our Flask app.

```python
from flask import Flask, render_template
app = Flask(__name__)

posts = [
    {
        'id': 1,
        'author': 'Your name',
        'title': 'Blog Post 1',
        'content': 'First post content',
        'date_posted': '4 March 2024'
    },
    {
        'id': 2,
        'author': 'Your name',
        'title': 'Blog Post 2',
        'content': 'Second post content',
        'date_posted': '5 March 2024'
    }
]


# Route for the home page, which is where the blog posts will be shown
@app.route("/")
@app.route("/home")
def home():
    return render_template('home.html', posts=posts)


# Route for the about page
@app.route("/about")
def about():
    return render_template('about.html', title='About page')
```

#### Step 2: Creating Blog Template

Now, we'll create a template to display the blog posts:

```html
{% extends 'layout.html' %}
{% block content %}
  {% for post in posts %}
    <div class="article">
      <h2>{{ post.title }}</h2>
      <p>By {{ post.author }} on {{ post.date_posted }}</p>
      <p>{{ post.content }}</p>
    </div>
  {% endfor %}
{% endblock %}
```

#### Step 3: Running the Flask App

Finally, we'll run our Flask app to see the blog page by running `flask --app ...`. Remember to run in debug mode!

## Why Use a Database in a web app? <a class="anchor" id="why"></a>

When building a Flask application, using a database instead of hardcoding data or using spreadsheets offers several advantages:

- **Data Integrity**: provide mechanisms to enforce data integrity through constraints and relationships, ensuring that data remains consistent and accurate (e.g. we make sure a certain data type or size is entered).
- **Scalability**: designed to handle large volumes of data and concurrent user requests, making them suitable for applications with growing user bases.
- **Querying Flexibility**: there is already a language 'built-in' to create complex queries to retrieve, filter, and analyse data based on various criteria.
- **Concurrency Control**: mechanisms for managing concurrent access to data, preventing conflicts.

For example, let's consider a website with a blog: Storing blog posts and related data (user information, ) in a database allows for efficient retrieval, searching, and filtering of posts based on author, category, date, etc. It also facilitates the addition of features such as user comments, likes, and analytics, which would be challenging to implement with hardcoded data or spreadsheets.

### Example Database Structure: Blog Posts

For our blog posts situation, a database structure for storing the posts and related data might include the following tables:

1. **Posts Table**:

| post_id | title           | content                           | author_id | date_posted | category_id |
|---------|-----------------|-----------------------------------|-----------|-------------|-------------|
| 1       | First Post      | Content of the first blog post.   | 1         | 04-03-2024  | 2           |
| 2       | Second Post     | Content of the second blog post.  | 2         | 05-03-2024  | 2           |

2. **Authors Table**:

| author_id | name      | email             | bio                            |
|-----------|-----------|-------------------|--------------------------------|
| 1         | John Doe  | john@example.com  | Lorem ipsum dolor sit amet...  |
| 2         | Jane Smith| jane@example.com  | Nulla vitae elit libero...     |

3. **Categories Table**:

| category_id | name     | description            |
|-------------|----------|------------------------|
| 1           | Tech     | In this category, we ..|
| 2           | Lifestyle| The lifestyle catego...|

This database structure allows us to efficiently manage and query blog posts, authors, and categories, facilitating the development of a robust and scalable blog application.

### Querying the Database

Once the database is set up with the above structure, we would normally query it using SQL. For example, to retrieve all blog blog titles along with their authors and categories, we might use the following SQL query:

```sql
SELECT p.title, a.name AS author, c.name AS category
FROM Posts p
JOIN Authors a ON p.author_id = a.author_id
JOIN Categories c ON p.category_id = c.category_id;
```

The resulting table would look like this:

| title       | author     | category   |
|-------------|------------|------------|
| First Post  | John Doe   | Lifestyle  |
| Second Post | Jane Smith | Lifestyle  |

## Databases in Flask <a class="anchor" id="databases_flask"></a>

It turns out that to integrate a database into your Flask app, you do not need to know SQL - there are Python library that help you work with databases by using Python classes (rather than having to deal with the database tables directly) (aka in a 'Pythonic' way). 

Specifically, to integrate databases into flask apps, we can use a library called `Flask-SQLAlchemy`, based on another popular Python library called `SQLAlchemy`. `SQLAlchemy` is an SQL toolkit and Object-Relational Mapping (ORM) library for Python, which allows developers to work with relational databases in the above-mentioned 'Pythonic' way. SQLAlchemy supports various types of SQL databases, including SQLite, MySQL, PostgreSQL, and others. 

This last point underlines one of the advantages of Flask: it provides support for integrating various types of databases into web applications and it does not support any single one of them 'natively'. This means that we can choose a database that fits our application: `SQLAlchemy` allows us to switch between databases, without having to change the code in our app or change the way in which we are accessing the database. For example, we might want to start off with an SQLite databae (which does not require a server) and later -- to deploy an application that requires a server -- we could change to MySQL or PostgreSQL. 

Let us extend our Flask application to use SQLAlchemy and SQLite for database operations. First, let's install the necessary packages. In your command line, activate your virtual environment and install Flask's wrapper for the SQLAlchemy package by running 

`pip install Flask-SQLAlchemy`.

## Example continued: a blog post (now with database) <a class="anchor" id="blog_database"></a>

Now, let's modify our Flask application to use SQLAlchemy and SQLite. 

#### Step 1: Creating the database model
Copy and paste the following code into your app module (e.g. `flaskapp.py`):

```python
from flask import Flask, render_template
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime

# Creating your app instance
app = Flask(__name__)

# Configuring the SQLite database
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///site.db'

# Creating a SQLAlchemy database instance
db = SQLAlchemy(app)
app.app_context().push()


# Defining a model for blog posts ('models' are used to represent tables in your database).
class BlogPost(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    content = db.Column(db.Text, nullable=False)
    author = db.Column(db.String(50), nullable=False)
    date_posted = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)

    def __repr__(self):
        return f"BlogPost('{self.title}', '{self.date_posted}')"


# Route for the home page, which is where the blog posts will be shown
@app.route("/")
@app.route("/home")
def home():
    # Querying all blog posts from the database
    posts = BlogPost.query.all()
    return render_template('home.html', posts=posts)


# Route for the about page
@app.route("/about")
def about():
    return render_template('about.html', title='About page')
```

The `BlogPost` class serves as an abstraction of the blog posts stored in the database. In the context of SQLAlchemy, a 'model' represents the structure of a database table and allows us to interact with the data in an object-oriented manner.

By defining the `BlogPost` class, we're essentially mapping the attributes of a blog post (such as title, content, author, and date posted) to columns in the database table. This abstraction simplifies database operations and promotes code readability and maintainability.

#### Step 2: Add new Blog Posts to the Database

To create a new blog post and add it to the database, we first need to add the database according to the model we specified in the code of our main app module (eventually this would be placed in a different python module, to make your code more tidy). Then, we need to create a new instance of the `BlogPost` class, set its attributes (e.g., title, content, author, date posted), and then add it to the database session. Finally, the changes are committed to integrate the new blog post in the database.

Let us look at an example of how a blog post can be added to the database by using the command line. Note that eventually, data will be added to databases through user input (rather than on the command line like this)!

First, go to your command line and make sure you have activated the virtual environment you use for running flask apps and that you are currently in the root directory that contains your app module (e.g. `flaskapp.py`). Then type `python` so that you can write the following code to create the database 'skeleton'.

```python
from flaskapp import db
from flaskapp import BlogPost
db.create_all()
```

If you now look at your project directory, there is now a folder called `instances` and inside the database with the name `site.db` was created. The structure of the database has been set up, but there are no rows in the blog post table yet! Let's add a post to the database. Underneath `db.create_all()`, now write:

```python
new_post = BlogPost(title='New Post', content='This is a new blog post.', author='Your name')
db.session.add(new_post)
db.session.commit()
```

We can also look at the entries in our database by 'querying' the tables in the command line. Let's add another blog post and then look at a list of all the blog posts that are currently stored in the database. In the command line, run:

```python
new_post_2 = BlogPost(title='New Post 2', content='This is another blog post.', author='Your name')
db.session.add(new_post_2)
db.session.commit()
BlogPost.query.all()
```

The last command will return a list of all blog posts that are currently in the database. In the code above, you can see that the same command is being used in the code that is triggered when a user goes to the home directory on our app! If we now reload our app, we will see that this new blog post is being displayed. (We also now have to make a small change to our `home.html` template, replacing `post.date_posted` by `post.date_posted.strftime('%Y-%m-%d')`, to make sure our `datetime` variable is displayed as a date.)

#### Step 3: Create relationships between tables

We might want a better way to store user information (rather than duplicating this information within the row of each blog post). For this, we create a new `User` class, which will represent a table in our database called `user` (the tables that are represented by the classes are automatically created with the same name as the class, but with all small letters). In the following code, there is now a relationship between the `User` and `BlogPost` model, because users are authors of one or multiple posts (one-to-many relationship). 

To establish this relationship, we created a `posts` attribute in the `User` model, which is **not** a column but a `relationship` to the `BlogPost` model. It runs a query in the background on the table with the blog posts, retrieving any post written by the user with the same id. The `backref` parameter is similar to adding a column called `author` to the `BlogPost` model (which, as you can see, we have removed from the `BlogPost` model). To specify the user in the `BlogPost` model, we now added a column called `user_id` to it, which is how the relationshipo to the `User` model is created (through the `db.ForeignKey('user.id')` command).

we have removed the `author` column for the `BlogPost` class (aka the `blog_post` table) and are now only referring to the user who wrote the blog post by their `user_id`, which links to the `id` column in the new `User` class (aka `user` table). The posts written by a particular user are accessed through the `posts` attribute; this is not a column but an SQL query that runs and 

```python
# Defining a model for users
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    name = db.Column(db.String(20), nullable=False)
    posts = db.relationship('BlogPost', backref='author', lazy=True)

    def __repr__(self):
        return f"User('{self.name}', '{self.id}'')"


# Defining a model for blog posts ('models' are used to represent tables in your database).
class BlogPost(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    content = db.Column(db.Text, nullable=False)
    # author = db.Column(db.String(50), nullable=False)
    date_posted = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'), nullable=False)

    def __repr__(self):
        return f"BlogPost('{self.title}', '{self.date_posted}')"
```


Let us clear the current database and create a row in the user table:

```python
from flaskapp import db
from flaskapp import User, BlogPost
db.drop_all()
db.create_all()
user1 = User(name='Spongebob Squarepants')
db.session.add(user1)
db.session.commit()
```

If you reload your app now, nothing should show up, since you have only created a user and no blog post exists yet. You can also see this by running 

```python
user1.posts
```

which should return an empty list. Let us create a post by this user:

```python
user1.id
post1 = BlogPost(title='First post', content='Some text this person has written.', user_id=user1.id)
post2 = BlogPost(title='Another post', content='Some other content.', user_id=user1.id)
db.session.add(post1)
db.session.add(post2)
db.session.commit()
```

Re-running the `user1.posts` command should show you a list of these two posts. If you reload your app now, it should show the blog posts you have just added. Note that we also need to make a small change to the `home.html` template. We need to refer to the author of the post as `{{ post.author.name }}` instead of `{{ post.author }}`, since the latter refers to the entire `User` class, as defined through the `backref` parameter, when we defined the relationship. 

## Exercises <a class="anchor" id="exercises"></a>

### Exercise 1

Go through the steps above for the example WITH the database and get it running as a flask app.

### Exercise 2

Now, add an extra column to the users table that contains a short biography about the user. To do this, you need to update the `User` class accordingly and then go to your command line, open a python shell, drop the current database, and recreate it (like we did above after creating the `User` table in the first place). 

### Exercise 3

Include this short user biography above the content of each blog post by updating your `home.html` template.

### Exercise 4

To understand in what way the SQLAlchemy statements connect to the SQL queries you already know, you can `stringify` SQLAlchemy statements. Go to your command line and open a python shell by typing `python` and import the database connection and the two models you created, followed by importing the `select` and `text` methods from `sqlalchemy`:

```python
from flaskapp import db, User, BlogPost
from sqlalchemy import select, text
```

Now write the SQLAlchemy equivalent of selecting all columns from the BlogPosts table:

```python
statement = select(BlogPost)
```

To see exactly what SQL query this represents, type the following and see what happens (this is where you 'stringify' the SQLAlchemy:

```python
print(str(statement)) 
```

Have a look at the [documentation](https://docs.sqlalchemy.org/en/20/orm/queryguide/index.html) to see how SQL queries are written with the SQLAlchemy ORM. Use the guide to try the following statements, stringifying the statement each time:

1. select the title and the date posted from the blog post table
2. select the content for the post with id 2 (you'll have to use filtering)
3. select the title and the author name from the blog post table (note that for this you'll need to do a join)

### Exercise 5

With the last one of your statements from exercise 4, run the query on your database by running

```python
result = db.session.execute(statement)
result.all()
```

Now, re-do the the result that was printed after that last line, this time by writing the statement directly in SQL. For this, create a new variable called `statement_sql`, to which you assign your SQL query wrapped in the `text` wrapper that you imported from `sqlalchemy`. It should look something like this:

```python
statement_sql = text('this is where your SQL query goes')
```

You can then query your database with this statement exactly like you did just now with the `db.session.execute` statement, only replacing the statement variable. 

### Exercise 6

In your main flask app module, replace the line `posts=BlogPost.query.all()` by a direct SQL query to the database.